Overall Plan:
1. Get the compositions we care about in V-Cr-Ti space
2. Fit CE model on them 
3. Run MCMC on those compositions 
4. Sample 3-5 structures with comparable energy
5. Relax the supercells using CHGNet 
6. Create vacancies in those supercells 
7. Relax the supercell (letting the cell change)
8. Create defects for all nearest neighbors 
9. relax with keeping the volume fixed 
10. interpolate between start and end point
11. conduct neb 
12. save the energies for that site and neighbor 
13. for each structure, create histogram of the mean and std energies over all NNs 

# 1 Get all the compositions we care about in the V-Cr-Ti Space


## Libraries

In [1]:
import os
from pymatgen.core import Structure

## Load the perfect structures

In [3]:
# load the perfect structures 
start_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/V0_90625-Cr0_046875-Ti0_046875_initial.cif'
middle_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/V0_90625-Cr0_046875-Ti0_046875_middle.cif'
end_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/V0_90625-Cr0_046875-Ti0_046875_final.cif'

start = Structure.from_file(start_path)
middle = Structure.from_file(middle_path)
end = Structure.from_file(end_path)

/home/myless/.mambaforge/envs/chgnet/lib/python3.10/site-packages/pymatgen/io/cif.py:1186: UserWarning: The default value of primitive was changed from True to False in https://github.com/materialsproject/pymatgen/pull/3419. CifParser now returns the cell in the CIF file as is. If you want the primitive cell, please set primitive=True explicitly.
  warnings.warn(


 # 2. Create Vacancies from the Structures

## Libraries

In [22]:
import os
import json
import sys
import numpy as np
import random
from smol.io import load_work
from pymatgen.core.structure import Structure
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.outputs import Outcar
from ase.db import connect
from ase.io import write
from ase.visualize import view

sys.path.append('../Modules')
from defect_maker import make_defects, return_x_neighbors
from vasp_misc import *
# Function to load and sort the structure
def load_and_sort_structure(entry):
    return Structure.from_dict(entry.structure.as_dict()).get_sorted_structure()

def _read_contcar_direct(contcar_file):
    try:
        with open(contcar_file, 'r') as file:
            lines = file.readlines()
            # Read the lattice constant
            lattice_constant = float(lines[1].strip())
            # Read the lattice vectors
            lattice_vectors = [list(map(float, line.strip().split())) for line in lines[2:5]]
            # Read the elements and their counts
            #elements = lines[5].strip().split()
            elements = [element.split('_')[0] for element in lines[5].strip().split()]
            print(elements)
            element_counts = list(map(int, lines[6].strip().split()))
            # Create a list of species that matches the number of coordinates
            species = [element for element, count in zip(elements, element_counts) for _ in range(count)]
            # Read the coordinates
            coordinates = [list(map(float, line.strip().split())) for line in lines[8:8+sum(element_counts)]]
            # Convert the coordinates from direct to Cartesian
            cartesian_coordinates = [[sum(a*b for a, b in zip(coord, vector)) for vector in lattice_vectors] for coord in coordinates]
            # Create the structure
            contcar = Structure(lattice_vectors, species, cartesian_coordinates)
            return contcar
    except Exception as e:
        print(f"Error reading CONTCAR file {contcar_file}: {e}")
        raise e
        return None
    
from pymatgen.core.structure import Structure

def read_contcar_direct(contcar_file):
    try:
        with open(contcar_file, 'r') as file:
            lines = file.readlines()
            # Read the lattice constant
            lattice_constant = float(lines[1].strip())
            # Read the lattice vectors
            lattice_vectors = [list(map(float, line.strip().split())) for line in lines[2:5]]
            # Read the elements and their counts
            elements_line = lines[5].strip().split()
            if '/' in elements_line[0]:
                # Handle format where elements are followed by identifiers
                #elements = [element.split('/')[0] for element in elements_line]
                elements = [element.split('/')[0].rstrip('_pv').rstrip('_sv') for element in elements_line]

            else:
                # Handle format where elements are directly listed
                elements = elements_line
            print(elements)
            element_counts = list(map(int, lines[6].strip().split()))
            # Create a list of species that matches the number of coordinates
            species = [element for element, count in zip(elements, element_counts) for _ in range(count)]
            # coordinate type
            coord_type = lines[7].strip()
            if coord_type.startswith(('c','C')):
                cart = True
            elif coord_type.startswith(('d','D')):
                cart = False
            # Read the coordinates
            coordinates_start_index = 8
            coordinates = []
            for line in lines[coordinates_start_index:coordinates_start_index + sum(element_counts)]:
                parts = line.strip().split()
                coordinates.append(list(map(float, parts[:3])))

            # Convert the coordinates from direct to Cartesian
            #cartesian_coordinates = [
                #[sum(a*b for a, b in zip(coord, vector)) for vector in zip(*lattice_vectors)]
                #for coord in coordinates
            #]
            # Create the structure
            contcar = Structure(lattice_vectors, species, coords=coordinates, coords_are_cartesian=cart)
            return contcar
    except Exception as e:
        print(f"Error reading CONTCAR file {contcar_file}: {e}")
        raise e
        return None


# Function to find N target atoms without overlapping second nearest neighbors
def _find_target_atoms(structure, N, neighbor_distance=2):
    all_indices = list(range(len(structure)))
    random.shuffle(all_indices)
    target_atoms = []
    neighbor_sets = []

    print(f"All indices: {all_indices}")

    while all_indices and len(target_atoms) < N:
        index = all_indices.pop()
        #neighbors, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=neighbor_distance, alat=structure.lattice.a)
        neighbors = []
        for distance in range(1, neighbor_distance + 1):
            neighbors_distance, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=distance, alat=structure.lattice.a)
            neighbors.extend(neighbors_distance)
        print(f"Index: {index}, Neighbors: {neighbors}")

        if not any(set(neighbors).intersection(neighbor_set) for neighbor_set in neighbor_sets):
            target_atoms.append(index)
            neighbor_sets.append(set(neighbors))
            print(f"Selected target atom index: {index}")

    #return target_atoms if len(target_atoms) == N else None
    return target_atoms 

def find_target_atoms(structure, N, neighbor_distance=2, cutoff_distance=5):
    all_indices = list(range(len(structure)))
    random.shuffle(all_indices)
    target_atoms = []
    neighbor_sets = []

    print(f"All indices: {all_indices}")

    while all_indices and len(target_atoms) < N:
        index = all_indices.pop()
        neighbors = []
        for distance in range(1, neighbor_distance + 1):
            neighbors_distance, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=distance, alat=structure.lattice.a)
            neighbors.extend(neighbors_distance)
        
        print(f"Index: {index}, Neighbors: {neighbors}")

        if not any(set(neighbors).intersection(neighbor_set) for neighbor_set in neighbor_sets):
            # Check if the distance to all existing target atoms is greater than the cutoff distance
            if all(structure.get_distance(index, target_atom) > cutoff_distance for target_atom in target_atoms):
                target_atoms.append(index)
                neighbor_sets.append(set(neighbors))
                print(f"Selected target atom index: {index}")

    #return target_atoms if len(target_atoms) == N else None
    return target_atoms

def _randomly_pick_sites(structure, n):
    # Ensure that n is not greater than the number of sites in the structure
    if n > len(structure.sites):
        raise ValueError("The number of sites to pick cannot be greater than the number of sites in the structure.")
    
    # Randomly select n sites from the structure
    random_sites = random.sample(structure.sites, n)
    return random_sites

import random
from scipy.spatial.distance import cdist

def _randomly_pick_sites(structure, n, cutoff=1):
    # Ensure that n is not greater than the number of sites in the structure
    if n > len(structure.sites):
        raise ValueError("The number of sites to pick cannot be greater than the number of sites in the structure.")
    
    # Randomly select n sites from the structure
    random_sites = []
    while len(random_sites) < n:
        potential_site = random.choice(structure.sites)
        print("Potential site: ", potential_site.coords)
        print(cutoff * min(structure.lattice.abc))
        if all(cdist([potential_site.coords], [site.coords])[0][0] > cutoff * min(structure.lattice.abc) for site in random_sites):
            print("Found site")
            random_sites.append(potential_site)
    return random_sites

import random
import numpy as np
from pymatgen.core.structure import Structure

def randomly_pick_sites(structure, n, initial_cutoff=1.25, max_attempts=1000, reduction_factor=0.9):
    """
    Randomly selects a specified number of sites from a given structure.

    Args:
        structure (Structure): The structure from which to randomly select sites.
        n (int): The number of sites to randomly select.
        initial_cutoff (float, optional): The initial cutoff distance for site selection. Defaults to 1.25.
        max_attempts (int, optional): The maximum number of attempts to make for site selection. Defaults to 1000.
        reduction_factor (float, optional): The reduction factor for the cutoff distance after each unsuccessful attempt. Defaults to 0.9.

    Returns:
        list: A list of randomly selected sites from the structure.

    Raises:
        ValueError: If the number of sites to pick is greater than the number of sites in the structure.
    """
    # Ensure that n is not greater than the number of sites in the structure
    if n > len(structure.sites):
        raise ValueError("The number of sites to pick cannot be greater than the number of sites in the structure.")
    
    # Randomly select n sites from the structure with iterative reduction in cutoff
    random_sites = []
    cutoff = initial_cutoff
    while len(random_sites) < n:
        attempts = 0
        while len(random_sites) < n and attempts < max_attempts:
            potential_site = random.choice(structure.sites)
            if all(np.linalg.norm(np.array(potential_site.coords) - np.array(site.coords)) > cutoff * min(structure.lattice.abc) for site in random_sites):
                random_sites.append(potential_site)
            attempts += 1
        
        if len(random_sites) < n:
            cutoff *= reduction_factor
            random_sites = []  # Reset and try again with a reduced cutoff
    
    return random_sites

# Function to select a random neighbor
def select_random_neighbor(structure, target_atom_index, x_neighbor):
    neighbors, _ = return_x_neighbors(structure, target_atom_index, x_neighbor, structure.lattice.a)
    if neighbors:
        return random.choice(neighbors)
    return None

# Function to create and save structures with vacancies
def create_and_save_structures(entries, N, job_path, neighbor_distance=2, cutoff_distance=1.25):
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        #target_atoms = find_target_atoms(structure, N, neighbor_distance, cutoff_distance)
        target_atoms = randomly_pick_sites(structure, N, cutoff = cutoff_distance)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            print("Start defect made")
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue

            for x_neighbor in [1, 2, 3]:
                print("Neigbor distance: ", x_neighbor)
                vac_site = select_random_neighbor(structure, target_atom_index, x_neighbor)
                if vac_site is not None:
                    _, end_structure = make_defects(structure, target_atom_index, vac_site)
                    if end_structure is None:
                        print(f"Failed to create end structure for entry {k+1}, target atom {target_atom_index}, vac_site {vac_site}.")
                        continue

                    # Create directory and filenames
                    directory = os.path.join(job_path, f"structure_{k}_vac_site_{n}")
                    os.makedirs(directory, exist_ok=True)
                    print(f"Created directory: {directory}")

                    start_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_start.vasp")
                    end_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_end_site_{vac_site}.vasp")

                    # Write the structures to POSCAR files
                    Poscar(start_structure).write_file(start_filename)
                    print(f"Written start structure to {start_filename}")
                    Poscar(end_structure).write_file(end_filename)
                    print(f"Written end structure to {end_filename}")

def _create_start_structures(entries, N, job_path, cutoff_distance=1.25):
    start_structures = []
    removed_indexes = []
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        target_atoms = randomly_pick_sites(structure, N, cutoff = cutoff_distance)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue
            start_structures.append(start_structure)
            removed_indexes.append(target_atom_index)
    return start_structures, removed_indexes

from pymatgen.entries.computed_entries import ComputedStructureEntry
from monty.json import MontyEncoder, MontyDecoder
from pymatgen.io.vasp import Poscar

def create_start_structures(entries, N, job_path, cutoff_distance=1.25,supercell_scheme=True):
    computed_entries = []
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        target_atoms = randomly_pick_sites(structure, N, initial_cutoff= cutoff_distance)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            #print("On Target Atom: ", target_atom_index)
            target_atom_index = structure.index(target_atom_index)
            target_atom_composition = structure[target_atom_index].specie

            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue

            # Save the structure as a .vasp file
            #start_filename = os.path.join(job_path, f"structure_{k}_vac_site_{n}_start.vasp")
            if supercell_scheme:
                start_filename = os.path.join(job_path, f"supercell_gen{entry.data['generation']}_comp{entry.data['comp']}_struct{entry.data['struct']}_vac_site{n}_start.vasp")
            else:
                start_filename = os.path.join(job_path, f"supercell_gen{entry.data['generation']}_comp{entry.data['comp']}_vac_site{n}_start.vasp")
            Poscar(start_structure).write_file(start_filename)
            print(f"Written start structure to {start_filename}")

            # Create a ComputedStructureEntry and add it to the list
            data = {'generation': entry.data['generation'],
                    'comp': entry.data['comp'], 
                    'struct': entry.data['struct'], 
                    'vac_index' : target_atom_index, 
                    'vac_comp' : target_atom_composition, 
                    'perfect_structure' : entry.structure.as_dict()}
            computed_entry = ComputedStructureEntry(start_structure, energy=0, data=data)
            computed_entries.append(computed_entry)

    # Save the computed entries as a JSON file
    with open(os.path.join(job_path, 'computed_entries.json'), 'w') as f:
        json.dump(computed_entries, f, cls=MontyEncoder)

    print("Computed entries saved to JSON file.")

def create_end_structures(start_structures, removed_indexes, job_path):
    for k, (start_structure, target_atom_index) in enumerate(zip(start_structures, removed_indexes)):
        for n, x_neighbor in enumerate([1, 2, 3]):
            vac_site = select_random_neighbor(start_structure, target_atom_index, x_neighbor)
            if vac_site is not None:
                _, end_structure = make_defects(start_structure, target_atom_index, vac_site)
                if end_structure is None:
                    print(f"Failed to create end structure for entry {k+1}, target atom {target_atom_index}, vac_site {vac_site}.")
                    continue

                # Create directory and filenames
                directory = os.path.join(job_path, f"structure_{k}_vac_site_{n}")
                os.makedirs(directory, exist_ok=True)
                print(f"Created directory: {directory}")

                start_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_start.vasp")
                end_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_end_site_{vac_site}.vasp")

                # Write the structures to POSCAR files
                Poscar(start_structure).write_file(start_filename)
                print(f"Written start structure to {start_filename}")
                Poscar(end_structure).write_file(end_filename)
                print(f"Written end structure to {end_filename}")

def check_overlapping_atoms(structure, distance_threshold=0.4):
    """
    Check if a pymatgen structure has overlapping atoms.
    
    Parameters:
    structure (Structure): The pymatgen structure to check.
    distance_threshold (float): The distance threshold below which atoms are considered overlapping.
    
    Returns:
    bool: True if there are overlapping atoms, False otherwise.
    """
    distances = structure.distance_matrix
    num_atoms = len(structure)

    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            if distances[i, j] < distance_threshold:
                return True
    return False

def _print_min_distance(structure: Structure):
    min_distance = float('inf')

    for i in range(len(structure)):
        for j in range(i+1, len(structure)):
            distance = structure[i].distance(structure[j])
            min_distance = min(min_distance, distance)

    print(f"The minimum distance between any two atoms in the structure is: {min_distance}")

def print_min_distance(structure: Structure):
    min_distance = float('inf')
    atom1, atom2 = None, None

    for i in range(len(structure)):
        for j in range(i+1, len(structure)):
            distance = structure[i].distance(structure[j])
            if distance < min_distance:
                min_distance = distance
                atom1, atom2 = i, j

    print(f"The minimum distance between any two atoms in the structure is: {min_distance}")
    print(f"The atoms are at indexes {atom1} and {atom2}")
    print(f"The coordinates of the atoms are {structure[atom1].coords} and {structure[atom2].coords}")

## Create Vacancies

In [20]:
# create an entry from teh structure 
def create_entry(structure, data):
    return ComputedStructureEntry(structure, energy=0, data=data)

# create a list of entries from a list of structures
def create_entries(structures, data):
    return [create_entry(structure, dat) for structure, dat in zip(structures, data)]

# create a list of entries from a list of structures
def create_entries_from_files(structure_files, data):
    structures = [Structure.from_file(file) for file in structure_files]
    data = []
    for structure in structures:
        struc_data = {'generation' : 0, 'comp' : structure.composition.reduced_formula, 'struct' : 'Fixed_4096'} 
        data.append(struc_data)
    return create_entries(structures, data)

# get file list
#file_list = [f for f in os.listdir('../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/') if f.endswith('.cif')]
file_list = [start_path, middle_path, end_path]
entries = create_entries_from_files(file_list, {'generation': 0, 'comp': 'VCrTi', 'struct': 'Fixed_4096'})

/home/myless/.mambaforge/envs/chgnet/lib/python3.10/site-packages/pymatgen/io/cif.py:1186: UserWarning: The default value of primitive was changed from True to False in https://github.com/materialsproject/pymatgen/pull/3419. CifParser now returns the cell in the CIF file as is. If you want the primitive cell, please set primitive=True explicitly.
  warnings.warn(


In [21]:
print(entries[0].data)

{'generation': 0, 'comp': 'Ti3V58Cr3', 'struct': 'Fixed_4096'}


In [24]:
# create start structure from the entries
create_start_structures(entries, 5, '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/Start_Structures', cutoff_distance=1.25, supercell_scheme=False)

Processing entry 1/3...
Found target atoms for entry 1: [PeriodicSite: Ti788 (Ti) (6.144, 2.128, 6.951) [0.1875, 0.0625, 0.25], PeriodicSite: V4015 (V) (24.58, 21.28, 17.38) [0.9375, 0.625, 0.9375], PeriodicSite: V205 (V) (-14.75, 25.54, 23.46) [0.0, 0.75, 0.8125], PeriodicSite: V4048 (V) (20.89, 27.67, -24.33) [0.9375, 0.8125, 0.0], PeriodicSite: Cr177 (Cr) (-13.52, 23.41, -6.951) [0.0, 0.6875, 0.0625]]
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site0_start.vasp
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site1_start.vasp
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site2_start.vasp
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_s

## Relax Each Vacancy Structure

In [25]:
from pymatgen.entries.computed_entries import ComputedStructureEntry
# now lets make the computed entries from chgnet potential 
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer

def create_computed_entry_from_chgnet(structure, energy):
    composition = structure.composition
    return ComputedStructureEntry(composition=composition, energy = energy, structure=structure)

def relax_structures(supercells, potential_path,device='cpu',verbose=False,relax_cell=True):
    vac_pot = CHGNet.from_file(potential_path, use_device=device)
    entries = []
    relaxer = StructOptimizer(vac_pot, use_device=device)

    # If vcrti_generated_supercells is a dictionary, convert it to a list
    if isinstance(supercells, dict):
        supercells = list(supercells.values())
    print(len(supercells))
    for i, supercell in enumerate(supercells):
        if isinstance(supercell, dict):
            supercell = Structure.from_dict(supercell)
        relaxed_supercell = relaxer.relax(atoms=supercell, fmax=0.05, relax_cell=relax_cell, verbose=verbose)
        print("Finished Volumetric Relaxing Structure: ", i)
        final_result = relaxer.relax(atoms=relaxed_supercell['final_structure'], fmax=0.05, relax_cell=False, verbose=verbose)
        print("Final Energy: ", final_result['trajectory'].energies[-1])

        # make the computed entry
        entry = create_computed_entry_from_chgnet(supercell, final_result['trajectory'].energies[-1])
        entries.append(entry.as_dict())

    return entries

In [26]:
structure_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/Start_Structures'

# get the structures from each .vasp file in the directory
start_structures = [Structure.from_file(os.path.join(structure_path, f)) for f in os.listdir(structure_path) if f.endswith('.vasp')]

# load chgnet 
pot_path = '../Potentials/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'

relaxed_entries = relax_structures(start_structures[0], pot_path, device='cuda', verbose=True)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda:0
5
      Step     Time          Energy          fmax
FIRE:    0 12:01:09   -36443.100586        1.197845
FIRE:    1 12:02:04   -36444.775958        1.158352


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
